In [1]:
# import library's
import time
from selenium import webdriver
import csv
import urllib.request as req
import random
from datetime import datetime
import pathlib
from selenium.webdriver.common.by import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import os
import json
from urllib.parse import urlparse
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pathlib import Path
import sys
from googleapiclient.discovery import build
from google.cloud import storage
from PIL import Image
from seleniumwire import webdriver
import threading
import queue
import io

project_init=False
if not os.path.isfile('config.csv'):
    CONFIG_DF=pd.DataFrame(columns=["SUMRUSH_EMAIL","PROXY","GOOGLE_CUSTOM_SEARCH_ENGINE_ID","GOOGLE_CUSTOM_SEARCH_API_KEY","GCS_S3_BUCKET_NAME","GCS_CREDENTIALS_PATH"])    
    CONFIG_DF.to_csv('config.csv',index=False)
    project_init=True
else:
    CONFIG_DF=pd.read_csv('config.csv')

if project_init:   
    sys.exit('Config And Proxy Just Intilized')
 
with open('config/global_config.json', 'r') as file:
    CONFIG_DATA = json.load(file)

# GOOGLE_CUSTOM_SEARCH_API_KEY=CONFIG_DATA['GOOGLE_CUSTOM_SEARCH_API_KEY']
# GOOGLE_CUSTOM_SEARCH_ENGINE_ID=CONFIG_DATA['GOOGLE_CUSTOM_SEARCH_ENGINE_ID']

# GCS_S3_BUCKET_NAME=CONFIG_DATA['GCS_S3_BUCKET_NAME']
# GCS_CREDENTIALS_PATH=Path.cwd()+'/'+CONFIG_DATA['GCS_CREDENTIALS_PATH']


In [77]:
# functions from another script
def _get_gcs_public_url(bucket_name: str, object_name: str, secure: bool=True):
    scheme = "https" if secure else "http"
    return f"{scheme}://storage.googleapis.com/{bucket_name}/{object_name}"



def _upload_to_gcs(bucket_name: str, file_name: str, buffer: bytes,GCS_CREDENTIALS_PATH):
    client = storage.Client.from_service_account_json(GCS_CREDENTIALS_PATH)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_string(buffer)

    return _get_gcs_public_url(bucket_name, file_name)


def rtn_url(image_path,website,GCS_S3_BUCKET_NAME,GCS_CREDENTIALS_PATH):
    image = Image.open(image_path)

    with io.BytesIO() as buffer:
        image.save(buffer, format="PNG")
        
        return _upload_to_gcs(GCS_S3_BUCKET_NAME, website+str(time.time()).split('.')[0]+'.png', buffer.getvalue(),GCS_CREDENTIALS_PATH)


# Create custom search service



# Get estimated indexed page numbers using the google custom search api
def _get_estimated_indexed_pages(service, api_key, cx, domain):
    query = f"site:{domain}"
    response = service.cse().list(q=query, cx=cx, key=api_key).execute()
    if 'error' in response:
        if response['error']['code'] == 403 and response['error']['errors'][0]['reason'] == 'dailyLimitExceeded':
            raise Exception("Your daily quota has been exhausted.")
        else:
            raise Exception(f"An error occurred: {response['error']['message']}")

    return response['searchInformation']['totalResults']


def get_estimated_pages(domain_with_scheme,GOOGLE_CUSTOM_SEARCH_API_KEY,GOOGLE_CUSTOM_SEARCH_ENGINE_ID) :
    custom_search_service = build(
    "customsearch", "v1", developerKey=GOOGLE_CUSTOM_SEARCH_API_KEY
    )
    # Create custom search service
    return _get_estimated_indexed_pages(
        custom_search_service,
        GOOGLE_CUSTOM_SEARCH_API_KEY,
        GOOGLE_CUSTOM_SEARCH_ENGINE_ID,
        domain_with_scheme
    )








# import library's

In [4]:
# login system
def login():
    inp=input('Do you want to Start Login System(Y/N)? ')
    if inp.lower()=='y':
        options = webdriver.ChromeOptions() 
        if CONFIG_DATA['SHOW_Window_on_login'].lower()!='yes':
            options.add_argument('headless')

        driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.
        driver.implicitly_wait(10)
        print('Starting.....')

        # %%
        file_path = 'config/smush_account.txt'  # Replace 'your_file.txt' with the actual file path
        try:
            with open(file_path, 'r') as file:
                # Read the entire contents of the file
                file_contents = file.read()
                file_contents=file_contents.split('\n')
                for i in file_contents:
                    driver.get('https://www.semrush.com/')
                    try:
                        driver.find_element(By.CSS_SELECTOR, "#srf-search-bar")
                        print(email, 'Just Logged In a while ago')
                        continue
                    except:
                        pass
                        
                    try:
                        driver.find_element(By.CSS_SELECTOR, ".ch2-dialog-actions  .ch2-allow-all-btn").click()
                    except:
                        pass
                    email=i.split(':')[0]
                    password=i.split(':')[1]


                    if os.path.exists('smush_cookies/'+email+'_cookies.json'):
                        with open('smush_cookies/ijguk787duis@icznn.com_cookies.json', 'r') as file:
                            cookies = json.load(file)
                            for cookie in cookies:
                                driver.add_cookie(cookie)
                            driver.refresh()
                        try:
                            driver.find_element(By.CSS_SELECTOR, "#srf-search-bar")
                            print(email, 'logged in from before')
                            continue
                        except:
                            pass
                        

                    driver.find_element(By.CSS_SELECTOR, "#loginForm #email").send_keys(email)
                    driver.find_element(By.CSS_SELECTOR, "#loginForm #password").send_keys(password)
                    # driver.find_element(By.CSS_SELECTOR, "#loginForm button[type=submit]").click()
                    driver.execute_script("arguments[0].click()", driver.find_element(By.CSS_SELECTOR, "#loginForm button[type=submit]"))
                    time.sleep(10)
                    try:
                        driver.find_element(By.CSS_SELECTOR, 'button[data-test="header-menu__user"]')
                        f=True
                        pass
                    except Exception as e:
                        f=False
                        print(email, 'Is Bad Credentials',e)
                        pass
                    if f==True:
                        cookies = driver.get_cookies()
                        with open('smush_cookies/'+email+'_cookies.json', 'w') as file:
                            json.dump(cookies, file)
                        print(email, 'Is logged in succesfully')
                        driver.delete_all_cookies()
                    
            driver.quit()
        except FileNotFoundError:
            print(f"The file  was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
        inp=input('Do you want to Login Another Account (Y/N)? ')
        if inp.lower()=='y':
            login()
# login system

In [5]:
# check if file is csv or excel
def is_csv(filename):
    # Get the file extension from the filename
    file_extension = os.path.splitext(filename)[1].lower()

    # Check if the file extension is CSV or Excel
    if file_extension in ['.csv']:
        return True
    else:
        return False
# check if file is csv or excel

In [6]:
# built driver connector
def build_selenium_browser(cookie_file,cookie):
    rt={'status':False, 'browser':None,'info':None}
    pxy=False

    temp_driver_df_config=CONFIG_DF[CONFIG_DF['SUMRUSH_EMAIL']=='nawed.wahedi01@gmail.com']
    if temp_driver_df_config.empty:
        rt['info']='Account Configuration not set.'
        return rt
    else:
        account_config= temp_driver_df_config.iloc[0]
        if account_config['PROXY']:
            pxy=True
    options = webdriver.ChromeOptions() 
    options.add_argument("pageLoadStrategy=none")
    options.add_argument("--disable-network-throttling")
    options.add_argument("--disable-cpu-throttling")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options.add_argument("--disable-features=NetworkService")
    options.add_argument("--window-size="+str(CONFIG_DATA['Resolution-W'])+"x"+str(CONFIG_DATA['Resolution-H']))
    options.add_argument("--hide-scrollbars")
    if CONFIG_DATA['SHOW_Window'].lower()!='yes' and False:
        options.add_argument('headless')
    if pxy==True:
        while True:
            proxy_options = {
                'proxy': {
                    'http': 'socks5://'+account_config['PROXY'],
                    'https': 'socks5://'+account_config['PROXY'],
                    'no_proxy': 'localhost,127.0.0.1'
                }
            }
            driver = webdriver.Chrome(seleniumwire_options=proxy_options,options=options)
            driver.set_page_load_timeout(60)
            # configure proxy driver
            try:
                driver.get('https://ifconfig.me/ip')
                print('Account '+cookie+' , Proxy Ip:',driver.find_element(By.CSS_SELECTOR,'body').text)
                break
            except Exception as e:
                print(cookie,'Proxy Not Working ! ',e)
                inp=input('As Proxy Not working You want to Reconfigure or skip this account?  (Y Reconfigure /N Skip): ')
                if inp.lower()=='y':
                    inp=input('Press Enter if you reconfigured the Proxy')
                    return  build_selenium_browser(cookie_file,cookie)
                else:
                    rt['info']='Account Skiped due proxy not working '+cookie
                return rt

    else:
        driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.
        driver.set_page_load_timeout(60)
        # configure driver
        try:
            driver.get('https://ifconfig.me/ip')
            print('Account '+cookie+' , Proxy Ip:',driver.find_element(By.CSS_SELECTOR,'body').text())
        except:
            print(cookie,'Proxy Not Working ! ')
            rt['info']='Proxy not working '+cookie
            return rt
    
    try:
        driver.get('https://www.semrush.com/')
    except TimeoutException:
        driver.execute_script("window.stop();")
    # load semrush

    try:
        with open(cookie_file, 'r') as file:
            # Read the entire contents of the file
            cookies__ = json.load(file)
            for cookie__ in cookies__:
                driver.add_cookie(cookie__)
            driver.refresh()
        c_=0
        while True:
            if c_>0:
                driver.execute_script("window.stop();")
            c_+=1
            
            if c_>5:
                rt['info']='Bad Credential. Try Again ! '+cookie
                return rt
                break
            try:
                element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-test="header-menu__user"]'))
                )
                rt['status']=True
                rt['browser']=driver            
                print(cookie,' Logged in successfully.')
                rt['browser']={'driver':driver,'config':account_config}
                return rt
                break
                pass
            except TimeoutException as e:
                continue
            except Exception as e:
                print(cookie,'Bad Credential. Try Again ! ',e)
                rt['info']='Bad Credential. Try Again ! '+cookie
                return rt
            
                
    except FileNotFoundError:
        print(f"The file  was not found. Try again !")
        rt['info']='Bad Credential. Try Again ! '+cookie
        return rt
    except Exception as e:
        print(f"An error occurred: {e}")
        rt['info']='An error occurred: {e}" ! '+cookie
        return rt
    return rt


In [7]:
# account login threads management 
def threads_init():
    print('Initlizing Threads')
    folder_path = 'smush_cookies'  # Replace with the actual path to your folder

    # Get a list of all files in the folder
    file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    d=[]
    # Print the list of file names
    for file in file_list:
        thread_path=folder_path+'/'+file
        thread_name=file.replace('_cookies.json','')
        print('\n\nIntializing Tread',thread_name)
        d.append([thread_path,thread_name])


    threads = []
    results = queue.Queue()

    for i in d:
        thread = threading.Thread(target=lambda i=i: results.put(build_selenium_browser(i[0],i[1])))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Retrieve results from the queue
    accounts_with_driver=[]
    while not results.empty():
        result = results.get()
        if result['status']:
            accounts_with_driver.append(result['browser'])
        else:
            print('Tread Skipped Due To',result['info'])
    return accounts_with_driver

# thds=threads_init()
# threads_init()


In [23]:
# main calling
if __name__ == "__main__" or True:
# if False:
    login()
    # intilize data files
    print('Intializing Data Files.....')
    while True:
        filename = input('Write Your Data File Name(Usa1.xlsx): ')
        file_path='data_files/'+filename
        if os.path.isfile(file_path):
            break
        else:
            print(f"The file {file_path} does not exist. Try Again.")
            continue

    if is_csv(file_path):
        df=pd.read_csv(file_path)
    else:
        df=pd.read_excel(file_path)
    if not "Authority Score" in df.columns:
        df['Authority Score'] = None
    if not "Organic Search Traffic" in df.columns:
        df['Organic Search Traffic'] = None
    if not "ImageURL" in df.columns:
        df['ImageURL'] = None
    if not "Page View" in df.columns:
        df['Page View'] = None
    
    if is_csv(file_path):
        df.to_csv(file_path,index=False)
    else:
        df.to_excel(file_path,index=False)
    
    thds=threads_init()

    display(df)
    
    
    # intilize data files

Intializing Data Files.....


,First Name,Last Name,Title,Company,Company Name for Emails,Email,Email Status,Seniority,Corporate Phone,Other Phone,...,Email Bounced,Replied,Demoed,Number of Retail Locations,Apollo Contact Id,Apollo Account Id,Authority Score,Organic Search Traffic,ImageURL,Page View
0,Dan,Pellegrini,President & CEO,Cooperative Community Energy,Cooperative Community Energy,pilgrim@ccenergy.com,Verified,C suite,'+1 877-228-8700,NaN,...,False,False,False,1.0,64edfa1530f27600012fc921,NaN,NaN,NaN,NaN,NaN
1,Andy,Simons,"Associate Vice President, Business Innovation ...",Guideboat Company,Guideboat Company,asimons@guideboat.com,Verified,Vp,NaN,NaN,...,False,False,False,NaN,64aa386198d99c0001dad825,NaN,NaN,NaN,NaN,NaN
2,Elsa,Leung,CEO,"ELSA L, Inc",ELSA L,elsa@elsal.com,Verified,C suite,'+1 415-472-8388,NaN,...,False,False,False,NaN,64a3c52870c7400001abf3db,NaN,NaN,NaN,NaN,NaN
3,Sharon,Lehrer,Vice President,"Lehrer Designs, Inc.",Lehrer Designs,sharon@lehrerdesigns.com,Verified,Vp,'+1 415-461-2212,NaN,...,False,False,False,1.0,64ec7a65a659970001e3a7d9,NaN,NaN,NaN,NaN,NaN
4,Radhika,Gulhar,Class President Medical School,Project RISHI,Project RISHI,rgulhar@projectrishi.org,Verified,Vp,NaN,NaN,...,False,False,False,NaN,64ec7ad8e57f7c0001e23911,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Nancy,Law,Founder and CEO,Santa Barbara Company,Santa Barbara Company,nancy@santabarbaracompany.com,Verified,Founder,'+1 805-845-3700,NaN,...,False,False,False,1.0,64c4abadc25373000152d691,NaN,NaN,NaN,NaN,NaN
95,Wendy,Foster,Buyer/Owner,Wendy Foster Clothing Stores,Wendy Foster Clothing Stores,jenna@wendyfoster.com,Verified,Owner,'+1 805-565-1599,NaN,...,False,False,False,NaN,64a76a214f8c3400012b2cdc,NaN,NaN,NaN,NaN,NaN
96,John,Shalhoob,Co-Owner,Shalhoob Meat Co,Shalhoob Meat Co,john@shalhoob.com,Verified,Owner,'+1 805-963-7733,NaN,...,False,False,False,1.0,64a5847a5e4e9f00018043cf,NaN,NaN,NaN,NaN,NaN
97,Laura,Eltherington,Company Owner,Chaucer's Bookstore,Chaucer's Bookstore,laura@santabarbarabooks.com,Verified,Owner,'+1 805-682-6787,NaN,...,False,False,False,1.0,64c9aab42484cf0001abeec0,NaN,NaN,NaN,NaN,NaN


In [75]:
def get_the_scores(website,driver,config,row,index):
    try:
        website = urlparse(website).netloc
    except Exception as e:
        print("Issue: Unknown Website in Number {index} skiped, website:",website,e)

    if row['Organic Search Traffic']==True and row['Authority Score']==True:
        # print('Already Generated ! Skipping This one',index,website)
        return {'status':True,'info':'Already Exist This one '+str(index)+' '+str(website),'data':None}
    
    
    loaded=True
    try:
        driver.get("https://www.semrush.com/analytics/overview/?q="+website+"&searchType=domain")
    except TimeoutException:
        driver.execute_script("window.stop();")
    except TypeError:
        loaded=False

    if loaded==True:
        c___=0
        while True:
            if c___>5:
                return {'status':False,'info':'Error: '+str(e),'data':None}
            c___+=1
            try:
                element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-at="do-keywords-trend"]')))
                break
            except TimeoutException:
                continue
            except Exception as e:
                return {'status':False,'info':'Error: '+str(e),'data':None}
            
        
        driver.execute_script('''
    document.querySelectorAll('.srf-layout_sidebar,.srf-layoutheader,.srf-layoutnotification.srf-layoutskip-to-content, .srf-layoutsearch-panel [data-ui-name="ProductHead.Row"], [data-at="report-tabs"][data-ui-name], footer, .help-menu_button, header , [data-at="do-keywords-trend"], [data-at="do-serp-features"], #srf-search-bar, [data-at="top-line"], .srf-layout__sidebar, .help-menu__button').forEach(nod=>{
    nod.remove();
    })

    document.querySelectorAll('[data-at="organic"],[data-at="adwords"],[data-at="backlinks"]').forEach(nod=>{
    nod.parentNode.remove();
    })
    ''')
        screen_shot='screenshots/'+website+".png"
        time.sleep(CONFIG_DATA['Screenshot-Delay'])
        driver.save_screenshot(screen_shot)
        try:
            authority=driver.find_element(By.CSS_SELECTOR,'[data-ui-name="Flex"][data-at="do-summary-as"] [data-at="primary-data"] a span').text
        except:
            authority=0
        
        try:
            authority_=int(float(authority))
            authority=authority_
        except:
            pass
    
        try:
            organic=driver.find_element(By.CSS_SELECTOR,'[data-ui-name="Flex"][data-at="do-summary-ot"] [data-at="primary-data"] a span').text
        except:
            organic=0
        
        pg=get_estimated_pages('https://'+str(website),config['GOOGLE_CUSTOM_SEARCH_API_KEY'],config['GOOGLE_CUSTOM_SEARCH_ENGINE_ID'])
        try:
            pg=int(float(pg))
        except:
            pass
    else:
        authority=0
        organic=0
        pg=0
        screen_shot=rtn_url(screen_shot,website)
    return {'data':[authority,organic,pg,screen_shot],'index':index}
    


In [72]:
def get_the_scores_init(row,browser):
    index=row[0]
    website=row[1]['Website']
    driver=browser['driver']
    config=browser['config']
    return get_the_scores(website,driver,config,row[1],index)

In [76]:
thrds_count=len(thds)
df_rows_count=df.shape[0]
c=0
while c<df_rows_count:
    r=[]
    for i in range(0,thrds_count):
        r.append([c,df.loc[c]])
        c=c+1
    # list the brunch of rows 

    df_threads = []
    df_threads_results = queue.Queue()

    # Create and start multiple threads
    c__=0
    for i in r:
        df_thread = threading.Thread(target=lambda i=i: df_threads_results.put(get_the_scores_init(i,thds[c__])))
        df_threads.append(df_thread)
        df_thread.start()
        c__+=1

    # Wait for all threads to finish
    for df_thread in df_threads:
        df_thread.join()

    # Retrieve results from the queue
    
    while not df_threads_results.empty():
        result = df_threads_results.get()
        

    print("Account Threads Initlized!")

    break

Exception in thread Thread-66 (<lambda>):
Traceback (most recent call last):
  File "c:\Users\joy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\joy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\joy\AppData\Local\Temp\ipykernel_19412\3779103692.py", line 17, in <lambda>
  File "C:\Users\joy\AppData\Local\Temp\ipykernel_19412\3929416739.py", line 6, in get_the_scores_init
  File "C:\Users\joy\AppData\Local\Temp\ipykernel_19412\2732639770.py", line 63, in get_the_scores
TypeError: get_estimated_pages() missing 2 required positional arguments: 'GOOGLE_CUSTOM_SEARCH_API_KEY' and 'GOOGLE_CUSTOM_SEARCH_ENGINE_ID'
Exception in thread Thread-67 (<lambda>):
Traceback (most recent call last):
  File "c:\Users\joy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Use

Account Threads Initlized!
